In [ ]:
F

In [9]:
print('******************************************Timetable Scheduler*****************************************************')

******************************************Timetable Scheduler*****************************************************


In [10]:
#librarries
import random
from tabulate import tabulate

In [11]:
# Constants
initial_population = 100
course_count = 5
exam_hall_count = 2
time_slot_count = 3

# Initialization course list, exam halls, and time slots
courses = [f"C{i+1}" for i in range(course_count)]
exam_halls = [f"H{i+1}" for i in range(exam_hall_count)]
time_slots = [f"T{i+1}" for i in range(time_slot_count)]

# Define function to check if a classroom is free at a given time slot
def is_classroom_free(classroom, time_slot, solution):
    for slot in solution:
        if slot[1] == time_slot and slot[2] == classroom:
            return False
    return True

# Initialize classrooms with sizes
classrooms = {"classroom": 60, "large hall": 120}

# Initialize dictionaries to track schedules and assignments
professor_schedule = {}
room_schedule = {}
professor_courses = {f"P{i+1}": 0 for i in range(10)}  # Assuming 10 professors
section_courses = {f"S{i+1}": 0 for i in range(5)}     # Assuming 5 sections

# Define function to check if two time slots are on the same or adjacent days
def are_days_adjacent(slot1, slot2):
    day1 = int(slot1[1][1:])
    day2 = int(slot2[1][1:])
    return abs(day1 - day2) <= 1

# Define a list of lab courses
lab_courses = ["C1", "C3"]  # Example lab courses

# Constraints: C1 and C2 have 10 common students, C1 and C4 have 5 common students, etc.
constraints = [
    [courses[0], courses[1], 10],
    [courses[0], courses[3], 5],
    [courses[1], courses[4], 7],
    [courses[2], courses[3], 12],
    [courses[3], courses[4], 8]
]

# Map constraints for easy lookup
constraints_map = {}
for constraint in constraints:
    if constraint[0] not in constraints_map:
        constraints_map[constraint[0]] = {}
    constraints_map[constraint[0]][constraint[1]] = constraint[2]
    if constraint[1] not in constraints_map:
        constraints_map[constraint[1]] = {}
    constraints_map[constraint[1]][constraint[0]] = constraint[2]

# Print initial data
print("Constraints Map:", constraints_map)
print("Courses:", courses)
print("Exam Halls:", exam_halls)
print("Time Slots:", time_slots)
print("Constraints:", constraints)

# Define the fitness function
# Define the fitness function
def fitness_function(constraints, solution):
    # Initialize points and violations
    points = 0
    violations = []

    # Check constraints
    for course in courses:
        constraint_courses = constraints_map[course]
        for constraint_course in constraint_courses:
            constraint_course_solution = [x for x in solution if x[0] == constraint_course]
            course_solution = [x for x in solution if x[0] == course]

            if constraint_course_solution[0][1] == course_solution[0][1]:
                if ([course, constraint_course] and [constraint_course, course]) not in violations:
                    violations.append([course, constraint_course])
                points -= 1
            else:
                points += 2

    # Check additional constraints
    for alloc in solution:
        if alloc[1] == "T3":
            points -= 0.5
            violations.append([f"T3 allocation to {alloc[2]}"])

    time_hall_combinations = [[time_slot, exam_hall] for time_slot in time_slots for exam_hall in exam_halls]

    for schedule in solution:
        time_hall_combination = [schedule[1], schedule[2]]
        if time_hall_combination in time_hall_combinations:
            time_hall_combinations.remove(time_hall_combination)
        else:
            points -= 0.5
            violations.append([f"Multiple courses allocated to {time_hall_combination[1]} in {time_hall_combination[0]}"])

    # Check if a classroom is free at a given time slot
    for schedule in solution:
        if not is_classroom_free(schedule[2], schedule[1], solution):
            points -= 1
            violations.append([f"Classroom {schedule[2]} is not free at time slot {schedule[1]}"])

    # Check if two time slots are on the same or adjacent days
    for i in range(len(solution)):
        for j in range(len(solution)):
            if i != j and are_days_adjacent(solution[i], solution[j]):
                points -= 0.5
                violations.append([f"Time slots {solution[i][1]} and {solution[j][1]} are on the same or adjacent days"])

    # Check if a professor is assigned two lectures at the same time
    for professor in professor_schedule:
        professor_slots = [x[1] for x in solution if x[3] == professor]
        if len(set(professor_slots)) != len(professor_slots):
            points -= 1
            violations.append([f"Professor {professor} is assigned two lectures at the same time"])

    # Check if the same section is assigned to two different rooms at the same time
    for section in section_courses:
        section_slots = [x[1] for x in solution if len(x) >= 5 and x[4] == section]
        if len(set(section_slots)) != len(section_slots):
            points -= 1
            violations.append([f"Section {section} is assigned to two different rooms at the same time"])

    # Check if a room is assigned for two different sections at the same time
    for room in room_schedule:
        room_slots = [x[1] for x in solution if x[2] == room]
        if len(set(room_slots)) != len(room_slots):
            points -= 1
            violations.append([f"Room {room} is assigned for two different sections at the same time"])

    # Check if a professor teaches more than 3 courses
    for professor, courses_assigned in professor_courses.items():
        if courses_assigned > 3:
            points -= 1
            violations.append([f"Professor {professor} teaches more than 3 courses"])

    # Check if a section has more than 5 courses in a semester
    for section, courses_assigned in section_courses.items():
        if courses_assigned > 5:
            points -= 1
            violations.append([f"Section {section} has more than 5 courses in a semester"])

    # Check if lab lectures are conducted in two consecutive slots
    for course in lab_courses:
        course_slots = [x[1] for x in solution if x[0] == course]
        course_slots.sort()  # Sort slots to check consecutive
        for i in range(len(course_slots) - 1):
            if not are_days_adjacent(course_slots[i], course_slots[i+1]):
                points -= 1
                violations.append([f"Lab lectures for course {course} are not conducted in two consecutive slots"])

    return points, violations




# Function to get violations in a solution
def get_violations(constraints, solution):
    violations = []
    for course in courses:
        constraint_courses = constraints_map[course]
        for constraint_course in constraint_courses:
            constraint_course_solution = [x for x in solution if x[0] == constraint_course]
            course_solution = [x for x in solution if x[0] == course]

            if constraint_course_solution[0][1] == course_solution[0][1]:
                if ([course, constraint_course] and [constraint_course, course]) not in violations:
                    violations.append([course, constraint_course])

    for alloc in solution:
        if alloc[1] == "T3":
            violations.append([f"T3 allocation to {alloc[2]}"])

    time_hall_combinations = [[time_slot, exam_hall] for time_slot in time_slots for exam_hall in exam_halls]

    for schedule in solution:
        time_hall_combination = [schedule[1], schedule[2]]
        if time_hall_combination not in time_hall_combinations:
            violations.append([f"Multiple courses allocated to {time_hall_combination[1]} in {time_hall_combination[0]}"])

    return violations

# Function to generate a random solution
def generate_random_solution():
    solution = []
    for course in courses:
        solution.append([course, random.choice(time_slots), random.choice(exam_halls)])
    return solution

# Generate initial population
def generate_population(count):
    population = []
    for i in range(count):
        chromosome = generate_random_solution()
        population.append([chromosome, fitness_function(constraints, chromosome)])
    return population

# Tournament selection
def tournament_selection(population, tournament_size):
    tournament = []
    for i in range(tournament_size):
        tournament.append(random.choice(population))
    tournament.sort(key=lambda x: x[1][0], reverse=True)
    return tournament[0]

# Local search algorithm
def local_search(constraints, solution):
    violations = get_violations(constraints, solution)
    if violations == []:
        return solution

    fitness_score, _ = fitness_function(constraints, solution)

    for i in range(len(solution)):
        for j in range(len(solution)):
            solution[i][1], solution[j][1] = solution[j][1], solution[i][1]
            new_violations = get_violations(constraints, solution)
            if new_violations == []:
                return solution

            new_fitness_score, _ = fitness_function(constraints, solution)

            if new_fitness_score > fitness_score:
                return solution

            solution[i][1], solution[j][1] = solution[j][1], solution[i][1]

    return solution

# Initial population and fitness calculation
population = generate_population(initial_population)
population.sort(key=lambda x: x[1][0], reverse=True)
best_solution = population[0]
best_solution[1] = fitness_function(constraints, best_solution[0])

# Evolutionary algorithm
max_iterations = 1000
iterations = 1000

while iterations > 0:
    fitness_score, violations = fitness_function(constraints, best_solution[0])
    best_solution[1] = (fitness_score, violations)  # Assign both fitness score and violations

    if violations == []:
        print("Best Solution Found! in", max_iterations - iterations, "iterations")
        break

    local_best_solution = population[0]
    if local_best_solution[1][0] > best_solution[1][0]:  # Access fitness score as tuple element
        best_solution = local_best_solution

    tournament_size = 5
    parent1 = tournament_selection(population, tournament_size)
    parent2 = tournament_selection(population, tournament_size)

    if random.random() <= 0.8:
        crossover_point = random.randint(0, len(parent1[0]) - 1)
        child1 = parent1[0][:crossover_point] + parent2[0][crossover_point:]
        child2 = parent2[0][:crossover_point] + parent1[0][crossover_point:]

        if random.random() <= 0.1:
            gene = random.randint(0, len(child1) - 1)
            child1[gene][1] = random.choice(time_slots)
            child1[gene][2] = random.choice(exam_halls)
            child2[gene][1] = random.choice(time_slots)
            child2[gene][2] = random.choice(exam_halls)

        child1_fitness = fitness_function(constraints, child1)
        child2_fitness = fitness_function(constraints, child2)

        population.sort(key=lambda x: x[1][0], reverse=True)
        population[-1] = [child1, child1_fitness]
        population[-2] = [child2, child2_fitness]

    iterations -= 1

print("Best Solution:", best_solution[0], "| With Score:", best_solution[1][0])
best_solution[0] = local_search(constraints, best_solution[0])
print("Best Solution After Local Search:", best_solution[0], "| With Score:", best_solution[1][0])

Constraints Map: {'C1': {'C2': 10, 'C4': 5}, 'C2': {'C1': 10, 'C5': 7}, 'C4': {'C1': 5, 'C3': 12, 'C5': 8}, 'C5': {'C2': 7, 'C4': 8}, 'C3': {'C4': 12}}
Courses: ['C1', 'C2', 'C3', 'C4', 'C5']
Exam Halls: ['H1', 'H2']
Time Slots: ['T1', 'T2', 'T3']
Constraints: [['C1', 'C2', 10], ['C1', 'C4', 5], ['C2', 'C5', 7], ['C3', 'C4', 12], ['C4', 'C5', 8]]
Best Solution: [['C1', 'T3', 'H1'], ['C2', 'T3', 'H1'], ['C3', 'T1', 'H2'], ['C4', 'T2', 'H1'], ['C5', 'T1', 'H1']] | With Score: 1.5
Best Solution After Local Search: [['C1', 'T1', 'H1'], ['C2', 'T3', 'H1'], ['C3', 'T3', 'H2'], ['C4', 'T2', 'H1'], ['C5', 'T1', 'H1']] | With Score: 1.5


In [13]:
import random

# Constants
initial_population = 100
course_count = 5
exam_hall_count = 2
time_slot_count = 3

# Initialize course list, exam halls, and time slots
courses = [f"C{i+1}" for i in range(course_count)]
exam_halls = [f"H{i+1}" for i in range(exam_hall_count)]
time_slots = {
    "Morning": ["8:30", "10:00"],
    "Afternoon": ["14:30"]
}

 # Soft Constraint: Morning Theory Classes, Afternoon Lab Sessions

# Initialize classrooms with sizes and floor numbers
classrooms = {
    "classroom1": {"size": 60, "floor": 1},
    "classroom2": {"size": 60, "floor": 2},
    "large_hall1": {"size": 120, "floor": 1},
    "large_hall2": {"size": 120, "floor": 2}
}

# Initialize dictionaries to track schedules and assignments
professor_schedule = {}
room_schedule = {}
professor_courses = {f"P{i+1}": 0 for i in range(10)}  # Assuming 10 professors
section_courses = {f"S{i+1}": 0 for i in range(5)}     # Assuming 5 sections

# Soft Constraint: All the theory classes should be taught in the morning session and all the lab sessions should be done in the afternoon session.
theory_courses = ["C1", "C2", "C5"]  # Example theory courses
lab_courses = ["C3", "C4"]  # Example lab courses

# Soft Constraint: Teachers/students may be facilitated by minimizing the number of floors they have to traverse.
# That is, as much as possible, scheduled classes should be on the same floor for either party.
def get_classroom_on_same_floor(course, previous_assignment=None):
    floor = None
    if previous_assignment:
        floor = classrooms[previous_assignment]["floor"]
    possible_classrooms = [cls for cls, details in classrooms.items() if details["floor"] == floor]
    if len(possible_classrooms) == 0:
        return random.choice(list(classrooms.keys()))
    return random.choice(possible_classrooms)

# Soft Constraint: A class should be held in the same classroom across the whole week.
classroom_assignments = {course: get_classroom_on_same_floor(course) for course in courses}

# Define the fitness function
def fitness_function(constraints, solution):
    # Initialize points as negative sum of conflicts
    points = 0

    # Check constraints
    for course in courses:
        constraint_courses = constraints_map[course]
        for constraint_course in constraint_courses:
            constraint_course_solution = [x for x in solution if x[0] == constraint_course]
            course_solution = [x for x in solution if x[0] == course]

            if constraint_course_solution[0][1] == course_solution[0][1]:
                points -= 1  # Decrement points for conflict
            else:
                points += 2   # Increment points for no conflict

    # Check additional constraints
    for alloc in solution:
        if alloc[1] == "T3":
            points -= 0.5  # Decrement points for conflict

    time_hall_combinations = [[time_slot, exam_hall] for time_slot in time_slots for exam_hall in exam_halls]

    for schedule in solution:
        time_hall_combination = [schedule[1], schedule[2]]
        if time_hall_combination in time_hall_combinations:
            time_hall_combinations.remove(time_hall_combination)
        else:
            points -= 0.5  # Decrement points for conflict

    # Check if a classroom is free at a given time slot
    for schedule in solution:
        if not is_classroom_free(schedule[2], schedule[1], solution):
            points -= 1  # Decrement points for conflict

    # Check if two time slots are on the same or adjacent days
    for i in range(len(solution)):
        for j in range(len(solution)):
            if i != j and are_days_adjacent(solution[i], solution[j]):
                points -= 0.5  # Decrement points for conflict

    # Check if a professor is assigned two lectures at the same time
    for professor in professor_schedule:
        professor_slots = [x[1] for x in solution if x[3] == professor]
        if len(set(professor_slots)) != len(professor_slots):
            points -= 1  # Decrement points for conflict

    # Check if the same section is assigned to two different rooms at the same time
    for section in section_courses:
        section_slots = [x[1] for x in solution if len(x) >= 5 and x[4] == section]
        if len(set(section_slots)) != len(section_slots):
            points -= 1  # Decrement points for conflict

    # Check if a room is assigned for two different sections at the same time
    for room in room_schedule:
        room_slots = [x[1] for x in solution if x[2] == room]
        if len(set(room_slots)) != len(room_slots):
            points -= 1  # Decrement points for conflict

    # Check if a professor teaches more than 3 courses
    for professor, courses_assigned in professor_courses.items():
        if courses_assigned > 3:
            points -= 1  # Decrement points for conflict

    # Check if a section has more than 5 courses in a semester
    for section, courses_assigned in section_courses.items():
        if courses_assigned > 5:
            points -= 1  # Decrement points for conflict

    # Check if lab lectures are conducted in two consecutive slots
    for course in lab_courses:
        course_slots = [x[1] for x in solution if x[0] == course]
        course_slots.sort()  # Sort slots to check consecutive
        for i in range(len(course_slots) - 1):
            if not are_days_adjacent(course_slots[i], course_slots[i+1]):
                points -= 1  # Decrement points for conflict

    return points, []  # Return points and an empty list for violations

# Function to check if a classroom is free at a given time slot
def is_classroom_free(classroom, time_slot, solution):
    for slot in solution:
        if slot[1] == time_slot and slot[2] == classroom:
            return False
    return True

def are_days_adjacent(slot1, slot2):
    day1 = slot1[6]  # Extract the day portion of the time string
    day2 = slot2[10]  # Extract the day portion of the time string
    return day1 == day2 or (day1[:-3] == day2[:-3] and abs(int(day1[-2:]) - int(day2[-2:])) <= 1)



# Function to generate a random solution
def generate_random_solution():
    solution = []
    for course in courses:
        theory_lab = "Theory" if course in theory_courses else "Lab"
        section = random.choice(list(section_courses.keys()))
        section_strength = random.randint(20, 50)
        professor = random.choice(list(professor_courses.keys()))
        first_lecture_day = random.choice(list(time_slots.keys()))
        first_lecture_timeslot = random.choice(time_slots[first_lecture_day])
        first_lecture_room = classroom_assignments[course]
        first_lecture_room_size = classrooms[first_lecture_room]["size"]
        second_lecture_day = random.choice(list(time_slots.keys()))
        second_lecture_timeslot = random.choice(time_slots[second_lecture_day])
        second_lecture_room = classroom_assignments[course]
        second_lecture_room_size = classrooms[second_lecture_room]["size"]
        solution.append([course, theory_lab, section, section_strength, professor,
                         first_lecture_day, first_lecture_timeslot, first_lecture_room,
                         first_lecture_room_size, second_lecture_day, second_lecture_timeslot,
                         second_lecture_room, second_lecture_room_size])
    return solution

# Generate initial population
def generate_population(count):
    population = []
    for i in range(count):
        chromosome = generate_random_solution()
        population.append([chromosome, fitness_function(constraints, chromosome)])
    return population

# Tournament selection
def tournament_selection(population, tournament_size):
    tournament = []
    for i in range(tournament_size):
        tournament.append(random.choice(population))
    tournament.sort(key=lambda x: x[1][0], reverse=True)
    return tournament[0]

# Crossover operation
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation operation
def mutate(chromosome, mutation_rate=0.1):
    mutated_chromosome = []
    for gene in chromosome:
        if random.random() < mutation_rate:
            # Flip the bit
            mutated_gene = 'Morning' if gene == 'Afternoon' else 'Afternoon'
        else:
            mutated_gene = gene
        mutated_chromosome.append(mutated_gene)
    return mutated_chromosome

# Local search algorithm
def local_search(constraints, solution):
    violations = get_violations(constraints, solution)
    if violations == []:
        return solution

    fitness_score, _ = fitness_function(constraints, solution)

    for i in range(len(solution)):
        for j in range(len(solution)):
            solution[i][1], solution[j][1] = solution[j][1], solution[i][1]
            new_violations = get_violations(constraints, solution)
            if new_violations == []:
                return solution

            new_fitness_score, _ = fitness_function(constraints, solution)

            if new_fitness_score > fitness_score:
                return solution

            solution[i][1], solution[j][1] = solution[j][1], solution[i][1]

    return solution

# Initial population and fitness calculation
population = generate_population(initial_population)
population.sort(key=lambda x: x[1][0], reverse=True)
best_solution = population[0]
best_solution[1] = fitness_function(constraints, best_solution[0])

# Evolutionary algorithm
max_iterations = 1000
iterations = 1000

while iterations > 0:
    fitness_score, violations = fitness_function(constraints, best_solution[0])
    best_solution[1] = (fitness_score, violations)  # Assign both fitness score and violations

    if violations == []:
        print("Best Solution Found! in", max_iterations - iterations, "iterations")
        break

    local_best_solution = population[0]
    if local_best_solution[1][0] > best_solution[1][0]:  # Access fitness score as tuple element
        best_solution = local_best_solution

    tournament_size = 5
    parent1 = tournament_selection(population, tournament_size)
    parent2 = tournament_selection(population, tournament_size)

    if random.random() <= 0.8:
        child1, child2 = crossover(parent1[0], parent2[0])
        child1_fitness = fitness_function(constraints, child1)
        child2_fitness = fitness_function(constraints, child2)

        population.sort(key=lambda x: x[1][0], reverse=True)
        population[-1] = [child1, child1_fitness]
        population[-2] = [child2, child2_fitness]

    if random.random() <= 0.1:
        gene = random.randint(0, len(population[0][0]) - 1)
        population[0][0][gene] = mutate(population[0][0][gene])

        gene = random.randint(0, len(population[1][0]) - 1)
        population[1][0][gene] = mutate(population[1][0][gene])

    iterations -= 1

print("Best Solution:", best_solution[0], "| With Score:", best_solution[1][0])
best_solution[0] = local_search(constraints, best_solution[0])
print("Best Solution After Local Search:", best_solution[0], "| With Score:", best_solution[1][0])

from tabulate import tabulate

# Assuming best_solution is a list of lists where each inner list represents a course assignment
# For example: [['C1', 'Theory', 'S1', 30, 'P1', 'Morning', 'T1', 'classroom1', 60, 'Afternoon', 'T3', 'classroom1', 60], [...], ...]

headers = ["Course", "Theory/Lab", "Section", "Strength", "Professor", "Day1", "Slot1", "Room1", "Room1 Size", "Day2", "Slot2", "Room2", "Room2 Size"]

#output in a table
print(tabulate(best_solution[0], headers=headers, tablefmt="grid"))





Best Solution Found! in 0 iterations
Best Solution: [['C1', 'Theory', 'S1', 22, 'P9', 'Afternoon', '14:30', 'classroom2', 60, 'Morning', '8:30', 'classroom2', 60], ['C2', 'Theory', 'S3', 31, 'P6', 'Afternoon', '14:30', 'large_hall1', 120, 'Morning', '8:30', 'large_hall1', 120], ['C3', 'Lab', 'S4', 40, 'P8', 'Afternoon', '14:30', 'large_hall1', 120, 'Morning', '8:30', 'large_hall1', 120], ['C4', 'Lab', 'S4', 33, 'P7', 'Afternoon', '14:30', 'classroom2', 60, 'Morning', '10:00', 'classroom2', 60], ['C5', 'Theory', 'S3', 25, 'P4', 'Morning', '10:00', 'large_hall1', 120, 'Morning', '10:00', 'large_hall1', 120]] | With Score: -6.0
Best Solution After Local Search: [['C1', 'Lab', 'S1', 22, 'P9', 'Afternoon', '14:30', 'classroom2', 60, 'Morning', '8:30', 'classroom2', 60], ['C2', 'Theory', 'S3', 31, 'P6', 'Afternoon', '14:30', 'large_hall1', 120, 'Morning', '8:30', 'large_hall1', 120], ['C3', 'Theory', 'S4', 40, 'P8', 'Afternoon', '14:30', 'large_hall1', 120, 'Morning', '8:30', 'large_hall1', 